### PUC-RIO - Curso de Pós-Graduação em Ciência de Dados e Analytics
MVP da Sprint Engenharia de Dados 01_2025

Leonardo Fernando Piato – Abril de 2025

###  1.	**Descrição e Objetivo do Trabalho:**

Este projeto visa analisar a incidência de focos de incêndio no Brasil no período de 1998 a 2017, com informações de em quais meses e estados ocorreram tais eventos. Nosso principal objetivo é responder as seguintes questões:

i)	O número de focos de incêndio tem sido crescente ao longo dos anos?

ii)	Quais as épocas e meses tem maior incidência de incêndios?

iii)	Ocorreu alguma alteração nos meses de maior incidência de incêndio ao longo dos anos?

iv)	Quais estados e regiões tem maior frequência de focos de incêndio?

v)	Ao longo do tempo os estados com maiores focos de incêndio permaneceram os mesmos?


## 2.	Detalhamento e Coleta dos dados 

A base de dados foi acessada diretamente na plataforma Kaggle, que é reconhecida por hospedar conjuntos de dados organizados para diversas áreas de análise. 

Os dados disponibilizados no Kaggle foram, por sua vez, originados do Portal de Dados do Governo Brasileiro (dados.gov.br), que é uma fonte oficial de informação. Este portal fornece conjuntos de dados públicos e estruturados sobre diversos tópicos, incluindo informações relacionadas à preservação ambiental e aos incêndios florestais.

Este conjunto de dados relata o número de incêndios florestais no Brasil dividido por estados. A série abrange um período de aproximadamente 10 anos (1998 a 2017). Os dados foram obtidos do site oficial do governo brasileiro, na na seguinte URL:

http://dados.gov.br/dataset/sistema-nacional-de-informacoes-florestais-snif 

Esses dados são importantes uma vez que os incêndios florestais são um problema sério para a preservação de nossa fauna e flora. Entender a frequência de incêndios florestais em uma série temporal pode nos ajudar a compreender como prevenir tais incidentes.

Os dados têm boa qualidade e são bem-organizados. Na fase de tratamento da base verificaremos que quase nenhuma correção será necessária. Devemos enfatizar também a simplicidade da base, são apenas cinco colunas, com cabeçalho e tendo por volta de cinco mil linhas.

Cabe enfatizar que tais **dados são livres de licença**, como todos os outros disponíveis no Portal de Dados do Governo Federal.

Após o download dos dados no formato CSV por meio da plataforma Kaggle, os arquivos foram transferidos e armazenados na plataforma Databricks, que oferece um ambiente robusto e escalável para análise de dados. A escolha do Databricks como plataforma de armazenamento e análise foi motivada pela sua capacidade de processamento distribuído, integração com PySpark e suporte para exploração e manipulação de grandes volumes de dados.

Essa etapa garantiu a persistência dos dados e preparou o ambiente para as análises subsequentes, como transformações, detecção de padrões e visualizações. Toda a documentação do processo foi realizada para assegurar a rastreabilidade e transparência dos dados coletados.

Dessa forma, resumidamente temos:

•	Fonte de Dados: Kaggle como plataforma intermediária e Portal de Dados do Governo Brasileiro como origem oficial

•	Formato de Dados: Arquivo CSV

•	Armazenamento na Nuvem: Persistência no Databricks

Nosso primeiro passo é fazer as importações de bibliotecas que serão usadas: 



In [0]:
#Primeiro importar as bibliotecas necessárias para efetuar a análise
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pyspark.sql.functions import col, sum
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

Agora vamos importar nossa base e visualizá-la

In [0]:
# Localização do arquivo e definição do tipo
file_location = "/FileStore/tables/Fire_br/Fire_br.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# As opções são aplicação para arquivos CSV. Para demais tipos de arquivos, serão ignoradas.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Agora visualizamos os dados com o comando display

display(df.limit(30)) 

year,state,month,number,date
1998,Acre,Janeiro,0,1998-01-01
1999,Acre,Janeiro,0,1999-01-01
2000,Acre,Janeiro,0,2000-01-01
2001,Acre,Janeiro,0,2001-01-01
2002,Acre,Janeiro,0,2002-01-01
2003,Acre,Janeiro,10,2003-01-01
2004,Acre,Janeiro,0,2004-01-01
2005,Acre,Janeiro,12,2005-01-01
2006,Acre,Janeiro,4,2006-01-01
2007,Acre,Janeiro,0,2007-01-01


Agora vamos excluir a coluna date, pois os dias e meses nela estão errados (todos constam como mes e dia 01 como pudemos ver acima). Damos um novo comando display para confirmar que a coluna date foi removida.

In [0]:
# Remover a coluna "Date" do DataFrame
df = df.drop("Date")

# Confirmar que a coluna foi removida
df.printSchema()

display(df.limit(30)) 

root
 |-- year: string (nullable = true)
 |-- state: string (nullable = true)
 |-- month: string (nullable = true)
 |-- number: string (nullable = true)



year,state,month,number
1998,Acre,Janeiro,0
1999,Acre,Janeiro,0
2000,Acre,Janeiro,0
2001,Acre,Janeiro,0
2002,Acre,Janeiro,0
2003,Acre,Janeiro,10
2004,Acre,Janeiro,0
2005,Acre,Janeiro,12
2006,Acre,Janeiro,4
2007,Acre,Janeiro,0


Agora temos nosso dataset com ano, estado, mês e número de incêndios.

## 3.	Modelagem e catálogo de dados:

Nosso primeiro passo começamos carregando os dados no Databricks, subimos os dados CSV, configuramos para leitura do arquivo definindo que há um cabeçalho e o delimitador é a virgula e exibimos os dados em um formato acessível para análise. 


### Modelo Flat para o Data Lake

Com o modelo flat, todos os dados são mantidos em um único DataFrame ou arquivo no Data Lake, dessa forma teremos flexibilidade e simplicidade no armazenamento, estrutura ideal para grandes volumes de dados prontos para análises futuras.



### Linhagem dos Dados

Em nosso modelo flat em Data Lake, nossa linhagem dos dados pode ser descrita da seguinte forma:

•	Fonte dos dados: Kaggle e Portal de Dados do Governo Brasileiro

•	Transformações:
  - Remoção da coluna date devido a inconsistência nos dados (todos os meses e dias constavam como 01)

  - Verificamos a existência de valores nulos e outliers e não foi identificada discrepâncias na base

  * Persistência: Armazenamento na plataforma Databricks


Utlizamos o modelo flat uma vez que possui os seguintes benefícios:

1 – Simplicidade: Um único dataset facilita o acesso e análises.

2 – Flexibilidade: Permite armazenar dados estruturas e semiestruturados.

3 – Escalabilidade: Ideal para grandes volumes de dados

4 – Velocidade: Menor esforço em normalização e consultas complexas.

Todos esses passos serão feitos nos códigos abaixo.

Antes de tudo começamos com o **catálogo de dados**


In [0]:

# Criar sessão Spark
spark = SparkSession.builder.appName("CatalogoDeDados").getOrCreate()

# Definir o esquema da tabela
schema = StructType([
    StructField("Coluna", StringType(), True),
    StructField("Descrição", StringType(), True),
    StructField("Tipo de Dado", StringType(), True),
    StructField("Valores Esperados", StringType(), True)
])

# Dados do catálogo
data = [
    ("ano", "Ano de registro", "Inteiro", "1998 a 2017"),
    ("estado", "Nome do estado", "String", 'Ex.: "São Paulo"'),
    ("mês", "Nome do mês", "String", '"Janeiro" a "Dezembro"'),
    ("numero_incendios", "Quantidade de incêndios registrados no mês", "Inteiro", ">=0")
]

# Criar o DataFrame com os dados fornecidos
df_catalogo = spark.createDataFrame(data, schema=schema)

# Exibir o DataFrame
df_catalogo.show(truncate=False)

+----------------+------------------------------------------+------------+----------------------+
|Coluna          |Descrição                                 |Tipo de Dado|Valores Esperados     |
+----------------+------------------------------------------+------------+----------------------+
|ano             |Ano de registro                           |Inteiro     |1998 a 2017           |
|estado          |Nome do estado                            |String      |Ex.: "São Paulo"      |
|mês             |Nome do mês                               |String      |"Janeiro" a "Dezembro"|
|numero_incendios|Quantidade de incêndios registrados no mês|Inteiro     |>=0                   |
+----------------+------------------------------------------+------------+----------------------+



Começamos carregando os dados no Databricks, subimos os dados CSV, configuramos para leitura do arquivo definindo que há um cabeçalho e o delimitador é a virgula e exibimos os dados em um formato acessível para análise. 

In [0]:
# Carregar os dados CSV
df_flat = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/tables/Fire_br/Fire_br.csv")

# Remover a coluna "Date"
df_flat = df_flat.drop("Date")

# Ajustar nomes das colunas
df_flat = df_flat.select(
    col("Year").alias("ano"),
    col("State").alias("estado"),
    col("Month").alias("mes"),
    col("number").alias("numero_incendios")
)

# Exibir estrutura
df_flat.show()

+----+------+-------+----------------+
| ano|estado|    mes|numero_incendios|
+----+------+-------+----------------+
|1998|  Acre|Janeiro|             0.0|
|1999|  Acre|Janeiro|             0.0|
|2000|  Acre|Janeiro|             0.0|
|2001|  Acre|Janeiro|             0.0|
|2002|  Acre|Janeiro|             0.0|
|2003|  Acre|Janeiro|            10.0|
|2004|  Acre|Janeiro|             0.0|
|2005|  Acre|Janeiro|            12.0|
|2006|  Acre|Janeiro|             4.0|
|2007|  Acre|Janeiro|             0.0|
|2008|  Acre|Janeiro|             0.0|
|2009|  Acre|Janeiro|             0.0|
|2010|  Acre|Janeiro|             1.0|
|2011|  Acre|Janeiro|             0.0|
|2012|  Acre|Janeiro|             0.0|
|2013|  Acre|Janeiro|             0.0|
|2014|  Acre|Janeiro|             0.0|
|2015|  Acre|Janeiro|             1.0|
|2016|  Acre|Janeiro|            12.0|
|2017|  Acre|Janeiro|             0.0|
+----+------+-------+----------------+
only showing top 20 rows



Agora salvamos o modelo em Parquet, uma vez que é um formato de armazenamento muito eficiente por ser "colunar", o que facilita consultas analíticas e melhora a performance ao manipular grandes volumes de dados.

In [0]:
# Salvar o modelo flat em formato Parquet
df_flat.write.format("parquet").mode("overwrite").save("/mnt/datalake/modelo_flat")

Código para verificar o DataFrame final antes da carga


In [0]:
# Verificar o DataFrame antes do carregamento
df_flat.show()
df_flat.printSchema()

+----+------+-------+----------------+
| ano|estado|    mes|numero_incendios|
+----+------+-------+----------------+
|1998|  Acre|Janeiro|             0.0|
|1999|  Acre|Janeiro|             0.0|
|2000|  Acre|Janeiro|             0.0|
|2001|  Acre|Janeiro|             0.0|
|2002|  Acre|Janeiro|             0.0|
|2003|  Acre|Janeiro|            10.0|
|2004|  Acre|Janeiro|             0.0|
|2005|  Acre|Janeiro|            12.0|
|2006|  Acre|Janeiro|             4.0|
|2007|  Acre|Janeiro|             0.0|
|2008|  Acre|Janeiro|             0.0|
|2009|  Acre|Janeiro|             0.0|
|2010|  Acre|Janeiro|             1.0|
|2011|  Acre|Janeiro|             0.0|
|2012|  Acre|Janeiro|             0.0|
|2013|  Acre|Janeiro|             0.0|
|2014|  Acre|Janeiro|             0.0|
|2015|  Acre|Janeiro|             1.0|
|2016|  Acre|Janeiro|            12.0|
|2017|  Acre|Janeiro|             0.0|
+----+------+-------+----------------+
only showing top 20 rows

root
 |-- ano: integer (nullable = tru

## 4.	Etapa de Carga

Primeiro, certificamos que os dados estão organizados e estruturados adequadamente a partir do modelo de dados flat que criamos. Usaremos um formato otimizado para armazenar, Delta Lake, devido à sua eficiência no armazenamento e consulta.

Vamos carregar os dados no delta lake e verificarmos os dados persistidos em seguida.

Vantagens do Delta Lake

* ACID Transactions: Garantia de integridade nos processos de leitura e escrita.

* Versionamento dos Dados: Permite consultas a versões anteriores, caso necessário.

* Escalabilidade: Ideal para grandes volumes de dados em plataformas de nuvem como Databricks.


Carregar os dados em Delta Lake

In [0]:
# Especificar o caminho para armazenar os dados no Delta Lake
caminho_dados_delta = "/mnt/datalake/incendios_flat_delta"

# Salvar o DataFrame no formato Delta Lake
df_flat.write.format("delta").mode("overwrite").save(caminho_dados_delta)

# Confirmação
print(f"Dados salvos com sucesso em Delta Lake no caminho {caminho_dados_delta}")

Dados salvos com sucesso em Delta Lake no caminho /mnt/datalake/incendios_flat_delta


E verificamos os dados persistidos, isto é, confirmamos que os dados foram salvos corretamente no local e formato escolhido. Essa verificação é uma etapa importante para garantir que os dados armazenados no Data Lake (no caso, no formato Parquet) possam ser recuperados e utilizados sem problemas em etapas futuras de análise ou processamento.

In [0]:
# Carregar os dados persistidos em Delta Lake para validação
df_verificado_delta = spark.read.format("delta").load(caminho_dados_delta)

# Exibir os primeiros registros para confirmar a carga e que estão no formato que queremos.
df_verificado_delta.show()

+----+------+-------+----------------+
| ano|estado|    mes|numero_incendios|
+----+------+-------+----------------+
|1998|  Acre|Janeiro|             0.0|
|1999|  Acre|Janeiro|             0.0|
|2000|  Acre|Janeiro|             0.0|
|2001|  Acre|Janeiro|             0.0|
|2002|  Acre|Janeiro|             0.0|
|2003|  Acre|Janeiro|            10.0|
|2004|  Acre|Janeiro|             0.0|
|2005|  Acre|Janeiro|            12.0|
|2006|  Acre|Janeiro|             4.0|
|2007|  Acre|Janeiro|             0.0|
|2008|  Acre|Janeiro|             0.0|
|2009|  Acre|Janeiro|             0.0|
|2010|  Acre|Janeiro|             1.0|
|2011|  Acre|Janeiro|             0.0|
|2012|  Acre|Janeiro|             0.0|
|2013|  Acre|Janeiro|             0.0|
|2014|  Acre|Janeiro|             0.0|
|2015|  Acre|Janeiro|             1.0|
|2016|  Acre|Janeiro|            12.0|
|2017|  Acre|Janeiro|             0.0|
+----+------+-------+----------------+
only showing top 20 rows



## 5. Análise

- Iniciamos verificando a qualidade de nossos dados. Há outliers? Há valores nulos? Enfim, vamos verificar se nosso conjunto de dados apresentam problemas que podem impactar em nossa análise.

Iniciamos verificando a existência de nulos. Conforme tabela abaixo, não temos esse problema.

In [0]:
# Contar valores nulos por coluna
from pyspark.sql import functions as F
df_verificado_delta.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_verificado_delta.columns]).show()

+---+------+---+----------------+
|ano|estado|mes|numero_incendios|
+---+------+---+----------------+
|  0|     0|  0|               0|
+---+------+---+----------------+



Verificando duplicados:

In [0]:
# Verificar duplicatas no DataFrame
duplicados = df_verificado_delta.count() - df_verificado_delta.dropDuplicates().count()
print(f"Número de registros duplicados: {duplicados}")

Número de registros duplicados: 32


Vamos dar uma olhada nos duplicados para ver se é alguma inconsistência.

In [0]:
df_duplicados = df_verificado_delta.exceptAll(df_verificado_delta.dropDuplicates())
df_duplicados.show()

+----+-----------+---------+----------------+
| ano|     estado|      mes|numero_incendios|
+----+-----------+---------+----------------+
|1998|Mato Grosso|     Maio|             0.0|
|1998|        Rio|  Janeiro|             0.0|
|1998|        Rio|  Janeiro|             0.0|
|1998|Mato Grosso|    Abril|             0.0|
|1998|        Rio|Fevereiro|             0.0|
|1998|        Rio|Fevereiro|             0.0|
|1998|    Paraiba|     Maio|             0.0|
|2009|        Rio|    Abril|             1.0|
|2017|    Alagoas|  Janeiro|            38.0|
|1998|        Rio|     Maio|             0.0|
|1998|        Rio|     Maio|             0.0|
|2007|        Rio|     Maio|             2.0|
|1998|        Rio|    Mar�o|             0.0|
|1998|        Rio|    Mar�o|             0.0|
|1998|    Paraiba|    Mar�o|             0.0|
|2004|        Rio|    Abril|             3.0|
|1998|        Rio|    Abril|             0.0|
|1998|        Rio|    Abril|             0.0|
|2008|        Rio|     Maio|      

Analisando  os valores duplicados vemos que não tratam-se de falhas. De fato existem muitos meses, principalmente os chuvosos, que alguns estados tem zero focos de incendio reportados. Dessa forma, considerando o contexto, não trata-se de um problema e não exige quaisquer tratamentos.

Abaixo partimos para analisar a distribuição e consistência dos dados:

In [0]:
# Estatísticas descritivas gerais
df_verificado_delta.describe().show()

+-------+------------------+---------+--------+------------------+
|summary|               ano|   estado|     mes|  numero_incendios|
+-------+------------------+---------+--------+------------------+
|  count|              6454|     6454|    6454|              6454|
|   mean|2007.4617291602108|     null|    null|108.29316284474758|
| stddev| 5.746653559687127|     null|    null| 190.8122420779928|
|    min|              1998|     Acre|   Abril|               0.0|
|    max|              2017|Tocantins|Setembro|             998.0|
+-------+------------------+---------+--------+------------------+



Dessa forma verificamos que os dados estão bem consistentes. Não temos número de incêndios negativo, por exemplo, que claramente seria um erro de consistência, nem quantidade de linhas diferentes para cada atributo.

Assim, partimos para a análise. Lembrando nossa pergunta 1:


> i)	O número de focos de incêndio tem sido crescente ao longo dos anos? 


Para a primeira pergunta vamos fazer agrupamentos por ano:

In [0]:
# Agrupar os dados por ano e somar o número total de incêndios
df_grouped_by_year = df_flat.groupBy("ano").agg(sum("numero_incendios").alias("total_incendios"))

# Ordenar os anos em ordem crescente
df_grouped_by_year = df_grouped_by_year.orderBy("ano")

# Exibir resultados para validação
df_grouped_by_year.show()


+----+------------------+
| ano|   total_incendios|
+----+------------------+
|1998|20013.971000000005|
|1999|26882.820999999993|
|2000|27351.250999999997|
|2001|         29071.612|
|2002|           37390.6|
|2003|42760.674000000006|
|2004| 38453.16300000001|
|2005| 35004.96499999999|
|2006|         33832.161|
|2007| 33037.41300000001|
|2008|         29378.964|
|2009|         39117.178|
|2010| 37037.44900000001|
|2011| 34633.54499999999|
|2012| 40084.86000000001|
|2013| 35146.11799999999|
|2014|39621.183000000005|
|2015| 41208.29200000001|
|2016| 42212.22900000001|
|2017|36685.623999999996|
+----+------------------+



E plotar em gráficos usando o Plotly, biblioteca que nos permite plotar gráficos interativos de forma fácil.

Por uma limitação do github, vamos colocar o link do gráfico plotado também, uma vez que após exportar nosso notebook do databricks para o Github, os gráficos são suprimidos (uma vez que estavam sendo gerado no ambiente do Databricks)

In [0]:
import plotly.express as px

# Converter para Pandas
pandas_df_year = df_grouped_by_year.toPandas()

# Criar gráfico de linhas com marcadores
fig = px.line(pandas_df_year, x="ano", y="total_incendios",
              title="Evolução Anual do Número de Incêndios",
              labels={"ano": "Ano", "total_incendios": "Número de Incêndios"},
              markers=True)  # Adicionar marcadores nos pontos

# Forçar todos os anos a aparecerem no eixo X
fig.update_layout(
    xaxis=dict(
        tickmode="array",  # Mostrar todos os valores
        tickvals=pandas_df_year["ano"].tolist()  # Lista com todos os anos
    ),
    xaxis_title="Ano",
    yaxis_title="Número de Incêndios",
    title_font_size=18
)

# Exibir o gráfico
fig.show()

[LINK PARA O GRÁFICO](https://github.com/XuxaRP/EngDados_Leonardo_Piato/blob/main/Evolucao_Anual_Incendios.png)

Dessa forma podemos responder nossa pergunta em partes. De 1997 a 2003 tivemos um incremento significativo, passando de 20 mil focos para mais de 42 mil focos de incêndio anuais. De 2003 a 2008 tivemos um periodo de diminuição para, a partir de 2009, ter uma estabilidade. Assim, podemos resumir que tivemos um número crescente de 1998 até 2003, e a partir dai tivemos uma relativa estabilidade.

* Pergunta dois: 

> "ii)	Quais as épocas e meses tem maior incidência de incêndios?"

In [0]:
# Agrupar por mês e calcular o total de incêndios
df_grouped_by_month = df_flat.groupBy("mes").agg(sum("numero_incendios").alias("total_incendios"))

# Ordenar os meses na sequência correta
meses_em_portugues = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
                      'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# Converter para Pandas para facilitar o ordenamento
pandas_df_month = df_grouped_by_month.toPandas()
pandas_df_month['mes'] = pd.Categorical(pandas_df_month['mes'], categories=meses_em_portugues, ordered=True)
pandas_df_month = pandas_df_month.sort_values('mes')

# Exibir os resultados organizados
print(pandas_df_month)

          mes  total_incendios
8     Janeiro        47747.844
2   Fevereiro        30848.050
9       Abril        28188.770
11       Maio        34731.363
1       Junho        56010.675
4       Julho        92326.113
0      Agosto        88050.435
5    Setembro        58578.305
10    Outubro        88681.579
6    Novembro        85508.054
7    Dezembro        57535.480
3         NaN        30717.405


In [0]:
# Criar gráfico de linhas com os meses e total de incêndios
fig = px.line(
    pandas_df_month,
    x="mes",
    y="total_incendios",
    title="Incêndios ao Longo dos Meses",
    labels={"mes": "Mês", "total_incendios": "Número de Incêndios"},
    markers=True  # Adicionar marcadores em cada ponto da linha
)

# Personalizar o layout
fig.update_layout(
    xaxis_title="Mês",
    yaxis_title="Número de Incêndios",
    title_font_size=18
)

# Exibir o gráfico
fig.show()

[Link](https://github.com/XuxaRP/EngDados_Leonardo_Piato/blob/main/Incendio_ao_longo_dos_meses.png) para o gráfico 2 gerado no Databricks

Aqui temos respostas claras, com os meses de Julho, agosto, outubro e novembro sendo os períodos com mais incêndios. Isso contrasta com fevereiro, abril e maio, epoca com menos focos.

Assim vamos para a terceira pergunta:

> iii)	Ocorreu alguma alteração nos meses de maior incidência de incêndio ao longo dos anos?

Para isso agrupamos os dados mês a mês e encontramos o mês em cada ano com mais casos de incendio.

In [0]:
# Agrupar os dados por ano e mês, somando os incêndios
df_grouped_by_year_month = df_flat.groupBy("ano", "mes").agg(sum("numero_incendios").alias("total_incendios"))

# Ordenar por ano e mês para facilitar a análise
df_grouped_by_year_month = df_grouped_by_year_month.orderBy("ano", "mes")

# Exibir os resultados organizados
df_grouped_by_year_month.show()

+----+---------+------------------+
| ano|      mes|   total_incendios|
+----+---------+------------------+
|1998|    Abril|               0.0|
|1998|   Agosto|          3964.616|
|1998| Dezembro|2043.4070000000002|
|1998|Fevereiro|               0.0|
|1998|  Janeiro|               0.0|
|1998|    Julho|4357.7119999999995|
|1998|    Junho|          1352.201|
|1998|     Maio|               0.0|
|1998|    Mar�o|               0.0|
|1998| Novembro|          2710.098|
|1998|  Outubro|2572.9429999999998|
|1998| Setembro|3012.9939999999997|
|1999|    Abril|             717.0|
|1999|   Agosto|          3563.958|
|1999| Dezembro|2267.1110000000003|
|1999|Fevereiro|            1284.0|
|1999|  Janeiro|            1081.0|
|1999|    Julho|          3538.223|
|1999|    Junho|          1433.201|
|1999|     Maio|            1812.0|
+----+---------+------------------+
only showing top 20 rows



In [0]:
# Encontrar o mês com maior incidência de incêndios em cada ano
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("ano").orderBy(F.desc("total_incendios"))
df_top_month = df_grouped_by_year_month.withColumn("rank", F.rank().over(windowSpec)).filter(F.col("rank") == 1)

# Exibir os meses com maior incidência de incêndios
df_top_month.select("ano", "mes", "total_incendios").show()

+----+--------+------------------+
| ano|     mes|   total_incendios|
+----+--------+------------------+
|1998|   Julho|4357.7119999999995|
|1999|Setembro|          3754.192|
|2000|  Agosto|          6524.693|
|2001| Outubro| 5289.773999999999|
|2002|   Julho|          6825.932|
|2003|Novembro|          5793.796|
|2004| Outubro|          5806.559|
|2005|Novembro|          4986.782|
|2006|  Agosto|7338.1269999999995|
|2007|   Julho|          4815.995|
|2008|  Agosto| 5975.561000000001|
|2009| Outubro|6239.9800000000005|
|2010|   Julho|          5356.301|
|2011|   Julho|          5441.086|
|2012|Novembro|          5428.166|
|2013|  Agosto| 5855.928000000001|
|2014| Outubro|5512.8369999999995|
|2015| Janeiro|            4635.0|
|2016|   Julho|          7105.048|
|2017|Novembro| 6217.379000000001|
+----+--------+------------------+



In [0]:
# Converter para Pandas para criar o gráfico
pandas_df_year_month = df_grouped_by_year_month.toPandas()

# Garantir que a coluna 'mes' está como categoria com ordem correta
ordem_meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
pandas_df_year_month["mes"] = pd.Categorical(pandas_df_year_month["mes"], categories=ordem_meses, ordered=True)

# Reordenar os dados pelo mês
pandas_df_year_month = pandas_df_year_month.sort_values("mes")

# Criar gráfico de linhas
fig = px.line(
    pandas_df_year_month,
    x="mes",
    y="total_incendios",
    color="ano",
    title="Incêndios Mensais ao Longo dos Anos",
    labels={"mes": "Mês", "total_incendios": "Número de Incêndios", "ano": "Ano"}
)

# Personalizar layout
fig.update_layout(
    xaxis_title="Mês",
    yaxis_title="Número de Incêndios",
    title_font_size=18
)

# Exibir o gráfico
fig.show()

[LINK para o Gráfico 3](https://github.com/XuxaRP/EngDados_Leonardo_Piato/blob/main/Incendios_Mensais.png)

A análise desse gráfico aparentemente torna mais confusa a resposta da pergunta. Dessa forma, verificando pela tabela que agrupa em qual mês houve mais incêndio em cada ano, podemos verificar que há uma padrão com os meses de agosto, julho, outubro e novembro se alternando em serem os meses "campeão" de incêndio no ano. Esse padrão reforça a sazonalidade dos incêndios no Brasil, especialmente nas regiões mais vulneráveis. Esse ciclo pode estar ligado a fatores climáticos, como períodos de seca, combinados com práticas humanas, como queimadas agrícolas.

Podemos ver que em 2015 o mês de janeiro foi o com mais incêndios, claramente sendo um outlier com base nos padrões que vimos. Fazemos uma rápida pesquisa, de acordo com o [IMPE](http://www.inpe.br/noticias/noticia.php?Cod_Noticia=3822) a anomalia de janeiro de 2015, quando o mês registrou um número recorde de incêndios, foi atribuída a uma combinação de fatores climáticos e humanos. Segundo o INPE, o ano de 2014 foi excepcionalmente seco, o que deixou a vegetação mais suscetível a queimadas no início de 2015. Além disso, o aumento do desmatamento na Amazônia Legal no final de 2014 contribuiu para o uso de fogo como método de limpeza de áreas desmatadas.

Agora vamos ver  a incidencia por estado para responder a pergunta quatro:

> iv)	Quais estados e regiões tem maior frequência de focos de incêndio?

In [0]:
# Agrupar por estado e calcular o total de incêndios
df_grouped_by_state = df_flat.groupBy("estado").agg(sum("numero_incendios").alias("total_incendios"))

# Ordenar pelo total de incêndios (do maior para o menor)
df_grouped_by_state = df_grouped_by_state.orderBy("total_incendios", ascending=False)

# Exibir os estados com maior número de incêndios
df_grouped_by_state.show()

+--------------+------------------+
|        estado|   total_incendios|
+--------------+------------------+
|   Mato Grosso| 96246.02799999995|
|       Paraiba| 52435.91800000001|
|     Sao Paulo|51121.198000000004|
|           Rio|         45160.865|
|         Bahia| 44746.22599999999|
|          Piau|37803.746999999996|
|         Goias|37695.520000000004|
|  Minas Gerais| 37475.25800000001|
|     Tocantins|33707.884999999995|
|      Amazonas|30650.129000000008|
|         Ceara|         30428.063|
|      Maranhao|         25129.131|
|          Par�| 24512.14400000002|
|    Pernambuco|           24498.0|
|       Roraima|         24385.074|
|Santa Catarina|         24359.852|
|         Amapa|21831.575999999997|
|      Rondonia|         20285.429|
|          Acre|          18464.03|
|Espirito Santo|            6546.0|
+--------------+------------------+
only showing top 20 rows



In [0]:
# Converter os dados agrupados para Pandas
pandas_df_state = df_grouped_by_state.toPandas()

# Criar gráfico de barras com tons avermelhados
fig = px.bar(
    pandas_df_state,
    x="estado",
    y="total_incendios",
    title="Total de Incêndios por Estado",
    labels={"estado": "Estado", "total_incendios": "Número de Incêndios"},
    color="total_incendios",  # Basear a cor no valor
    color_continuous_scale=px.colors.sequential.Reds  # Usar tons avermelhados
)

# Personalizar o layout
fig.update_layout(
    xaxis_title="Estado",
    yaxis_title="Número de Incêndios",
    title_font_size=18
)

# Exibir o gráfico
fig.show()

[LINK para o Gráfico 4](https://github.com/XuxaRP/EngDados_Leonardo_Piato/blob/main/Total_por_estados.png)

Esse gráfico nos mostra claramente que no período analisando o Mato Grosso se destacou como o que mais sofreu com incêndios, ao passo que vemos que Espirito Santos, Alagoas, DF e Sergipe tiveram relativamente poucos problemas do tipo, indicando fatores climáticos favoráveis ou menor dependência de queimadas.

Por fim temos a última pergunta:

> v)	Ao longo do tempo os estados com maiores focos de incêndio permaneceram os mesmos?

Para isso vamos fazer algumas manipulações e plotar apenas os estados que possuem número mais robustos, para nosso gráfico ter sua análise simplificada.

In [0]:
# Agrupar os dados por estado e ano, somando os incêndios
df_grouped_by_state_year = df_flat.groupBy("estado", "ano").agg(F.sum("numero_incendios").alias("total_incendios"))

# Ordenar por ano e número de incêndios
df_grouped_by_state_year = df_grouped_by_state_year.orderBy("ano", "total_incendios", ascending=[True, False])

# Exibir os resultados organizados
df_grouped_by_state_year.show()

+----------------+----+------------------+
|          estado| ano|   total_incendios|
+----------------+----+------------------+
|       Sao Paulo|1998|            3196.0|
|     Mato Grosso|1998|          2401.389|
|           Ceara|1998|1612.0140000000001|
|            Piau|1998|          1493.577|
|    Minas Gerais|1998|          1233.074|
|           Bahia|1998|1224.6870000000001|
|             Rio|1998|            1149.0|
|        Amazonas|1998|             946.0|
|        Rondonia|1998| 916.0070000000001|
|       Tocantins|1998|           912.634|
|         Paraiba|1998|             816.0|
|      Pernambuco|1998|             767.0|
|           Goias|1998| 750.1189999999999|
|            Acre|1998|             730.0|
|            Par�|1998|           715.586|
|        Maranhao|1998|           365.884|
|           Amapa|1998|             278.0|
|  Espirito Santo|1998|             218.0|
|Distrito Federal|1998|             103.0|
|         Alagoas|1998|              86.0|
+----------

In [0]:
# Encontrar os estados com maior número de incêndios em cada ano

windowSpec = Window.partitionBy("ano").orderBy(F.desc("total_incendios"))
df_top_states = df_grouped_by_state_year.withColumn("rank", F.rank().over(windowSpec)).filter(F.col("rank") <= 3)

# Exibir os 3 estados mais afetados por ano
df_top_states.select("ano", "estado", "total_incendios").show()

+----+-----------+------------------+
| ano|     estado|   total_incendios|
+----+-----------+------------------+
|1998|  Sao Paulo|            3196.0|
|1998|Mato Grosso|          2401.389|
|1998|      Ceara|1612.0140000000001|
|1999|  Sao Paulo| 4249.210999999999|
|1999|    Paraiba|            3437.0|
|1999|Mato Grosso|          2979.399|
|2000|  Sao Paulo|            4128.0|
|2000|Mato Grosso|          3897.266|
|2000|    Paraiba|            2804.0|
|2001|Mato Grosso|          3088.075|
|2001|  Sao Paulo|            2926.0|
|2001|      Bahia|          2427.651|
|2002|Mato Grosso|3665.5029999999992|
|2002|  Sao Paulo|            3539.0|
|2002|        Rio|            2945.0|
|2003|Mato Grosso| 6939.853999999999|
|2003|        Rio|          3438.419|
|2003|  Sao Paulo|            3306.0|
|2004|Mato Grosso|          4811.176|
|2004|        Rio|            3437.0|
+----+-----------+------------------+
only showing top 20 rows



In [0]:
# Converter para Pandas
pandas_df_top_states = df_top_states.toPandas()

# Criar gráfico de linhas para os estados mais afetados
fig = px.line(
    pandas_df_top_states,
    x="ano",
    y="total_incendios",
    color="estado",
    title="Evolução dos Estados com Maiores Focos de Incêndios ao Longo dos Anos",
    labels={"ano": "Ano", "total_incendios": "Número de Incêndios", "estado": "Estado"}
)

# Personalizar layout
fig.update_layout(
    xaxis_title="Ano",
    yaxis_title="Número de Incêndios",
    title_font_size=18
)

fig.show()

[Link para o Gráfico 5](https://github.com/XuxaRP/EngDados_Leonardo_Piato/blob/main/Evolucao_Por_estado.png)

Essa última análise traz insights interessantes. A primeira é, que de um modo geral, Mato Grosso é e permaneceu quase a série toda como o estado que mais sofre com o problema dos incêndios florestais, essa consistência em ser o estado mais afetado, podemos inferir que é devido as características específicas da região, como clima ou extensão de áreas florestais, e as lacunas nos dados de alguns estados outros estados.

Além disso podemos ver que em alguns estados as séries estão incompletas, como em São Paulo cujos dados vão até 2014, ou a Bahia, que se inicia apenas em 2001.



## 6. Autoanálise

De um modo geral, todas as perguntas foram respondidas. Acredito que apenas a pergunta "Ocorreu alguma alteração nos meses com maior incidência de incêndios ao longo do tempo?" apresentou maiores dificuldades para resolver. Por tratar-se de um período relativamente longo, é um dado difícil de plotar de forma não poluída. Dessa forma, a análise fica prejudicada. Contudo, ao analisar a tabela, é possível responder o questionamento.

Optei por usar o Python, pois foi a linguagem que me habituei ao longo das últimas três sprints e pareceu ter mais recursos disponíveis para a sessão de análise, devido às bibliotecas poderosas, como Pandas.

Acredito que o Databricks demonstrou ser uma ferramenta adequada à solução. Apresentou boa compatibilidade com usuários habituados ao Jupyter Notebook e um desempenho bastante razoável.

Imagino que, caso eu queira me aprofundar na atuação com data lakes ou warehouses, seja imprescindível aprimorar meus conhecimentos em SQL e também me habituar a trabalhar com dados "sujos". Isso contrasta com os dados deste projeto, que aparentemente já estavam "tratados".